In [1]:
from xlsxwriter.workbook import Workbook
import re
import pandas as pd 
from tqdm import tqdm

In [60]:
df = pd.read_excel('edgar2000-2024.xlsx')

# df['keyword'] is in the shape of a string '['keyword1', 'keyword2', 'keyword3']'
# we want extract each keyword and turn it into a list
def extract_keywords(row):
    keywords = re.findall(r"'(.*?)'", row)
    return keywords
df['keyword'] = df['keyword'].apply(extract_keywords)




In [56]:
# 创建Excel对象
workbook = Workbook('highlight.xlsx')
# open sheet1
worksheet = workbook.add_worksheet('Sheet1')

color = workbook.add_format({'color': 'red', 'bold': True})

for r in range(len(df)):
    doc = df.loc[r, 'doc']
    keywords = df.loc[r, 'keyword']

    try:
        pattern = ''
        for key in keywords:
            if key != keywords[-1]:
                pattern +=  f'({re.escape(key)})|'
            else:
                pattern +=  f'({re.escape(key)})'

        pattern = re.compile(pattern)

        format_ls = pattern.split(doc)

        # 去除list中的none
        format_ls = [i for i in format_ls if i]


        for i in range(len(format_ls)-1, -1, -1):
            if pattern.fullmatch(format_ls[i]):
                format_ls.insert(i, color)  # Prefix the word with the format
    except:
        format_ls = [color, doc]


    # 写入单元格
    row, col = r+1, 1
    worksheet.write_rich_string(row, col, *format_ls)
workbook.close()



/opt/anaconda3/envs/learn/lib/python3.9/site-packages/xlsxwriter/worksheet.py:1344: UserWarning: You must specify more than 2 format/fragments for rich strings. Ignoring input in write_rich_string().
  warn(


In [75]:

r = 1
doc = df.loc[r, 'doc']
keywords = df.loc[r, 'keyword']

pattern = ''
for key in keywords:

    if key != keywords[-1]:
        pattern +=  f"({re.escape(key)})|"
    else:
        pattern +=  f"({re.escape(key)})"

pattern = re.compile(pattern)

format_ls = pattern.split(doc)

# 去除list中的none
format_ls = [i for i in format_ls if i]


for i in range(len(format_ls)-1, -1, -1):
    if pattern.fullmatch(format_ls[i]):
        format_ls.insert(i, color)  # Prefix the word with the format

format_ls





['Pension Settlement Charge\n\u200b\nDuring the first quarter of fiscal 2024, we settled all future obligations under our frozen U.S. defined benefit retirement plan. The settlement included a combination of lump-sum payments to participants who elected to receive them and the transfer of the remaining benefit obligations to a third-party insurance company under a group annuity contract. As a result of the settlement, in the six-month period ended November 30, 2023, we recognized a non-cash, pre-tax pension settlement charge of $26.7 million ($16.1 million after-tax) related to the accelerated recognition of all unamortized net actuarial losses in Accumulated other comprehensive loss.']

In [67]:
# 把所有keyword中双斜杠改成单斜杠
def change_slash(row):
    keywords = re.sub(r'\\', r'、', row)
    return keywords